In [1]:
%load_ext autoreload
%autoreload 2

import logging
from functools import partial

import pandas as pd

from etf_scraper import ETFScraper, load_listings
from etf_scraper.utils import get_interval_query_dates
from etf_scraper.storage import query_range, save_func

logger = logging.getLogger()
logger.setLevel(logging.INFO)

query_dates = get_interval_query_dates('2020-01-01', '2023-01-20', False, False)
save_func_ = partial(save_func, out_dir="/tmp/out/")

listings = load_listings()

!mkdir /tmp/out/

mkdir: cannot create directory ‘/tmp/out/’: File exists


In [3]:
query_tickers = listings.ticker.unique()
query_dates = [None]
len(query_tickers)

1240

In [4]:
a = query_range(query_dates, query_tickers, ETFScraper(), save_func_)

INFO:etf_scraper.api:Querying for EIDO holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BKTSX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for IWV holdings as of latest from IShares
INFO:etf_scraper.api:Querying for IXC holdings as of latest from IShares
INFO:etf_scraper.api:Querying for MUB holdings as of latest from IShares
INFO:etf_scraper.api:Querying for ISTB holdings as of latest from IShares
INFO:etf_scraper.api:Querying for MCHI holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BSPPX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BSPIX holdings as of latest from IShares
INFO:etf_scraper.api:Querying for BMED holdings as of latest from IShares
INFO:etf_scraper.scrapers:Querying IShares for EIDO holdings as of None
INFO:etf_scraper.scrapers:Querying IShares for BKTSX holdings as of None
INFO:etf_scraper.scrapers:Querying IShares for IWV holdings as of None
INFO:etf_scraper.scrapers:Querying IShares f

In [5]:
df = pd.DataFrame(a).T.rename_axis(index=['ticker', 'date']).reset_index()
df_ = df.merge(listings, on='ticker')
df['error_class'].value_counts(dropna=False)

NaN                      743
ValueError               257
InvalidParameterError    191
KeyError                  38
EmptyDataError             6
HTTPError                  5
Name: error_class, dtype: int64

In [7]:
df_[~df_['error'].isna()][['provider', "asset_class"]].value_counts(dropna=False).sort_index()

provider  asset_class   
IShares   Commodity           3
          Equity             39
          Fixed Income      126
          Multi Asset         9
          Real Estate         3
Invesco   NaN                86
SSGA      Alternative         2
          Fixed Income       38
          NaN                69
Vanguard  Balanced Funds     10
          Bond Funds         55
          Stock Funds        57
dtype: int64

In [13]:
df_[
    (~df_['error'].isna())&(df_['provider']=='IShares')&(df_['asset_class']=='Equity')
    &(df_['fund_type']=='ETF')
    # &(df_['ticker']=='SMMD')
    ]#['error'].iloc[0]

,ticker,date,save_path,n_holdings,error,error_class,fund_name,inception_date,cusip,isin,...,subasset_class,country,region,product_url,product_id,net_assets,fund_type,provider,benchmark,exchange
184,SMMD,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Russell 2500 ETF,2017-07-06,46435G268,US46435G2681,...,Mid/Small Cap,United States,North America,https://www.ishares.com/us/products/288024/ish...,288024,5.773183e+08,ETF,IShares,NaN,NaN
198,HEWU,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI United Kingdom ETF,2015-06-29,46435G862,US46435G8621,...,Large/Mid Cap,United Kingdom,Europe,https://www.ishares.com/us/products/273763/ish...,273763,1.430883e+07,ETF,IShares,NaN,NaN
200,HEWC,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI Canada ETF,2015-06-29,46435G706,US46435G7060,...,Large/Mid Cap,Canada,North America,https://www.ishares.com/us/products/273775/ish...,273775,1.457859e+07,ETF,IShares,NaN,NaN
201,HAWX,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI ACWI ex U.S. ETF,2015-06-29,46435G847,US46435G8472,...,Large/Mid Cap,Broad,Global,https://www.ishares.com/us/products/273768/ish...,273768,1.595968e+08,ETF,IShares,NaN,NaN
202,HSCZ,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI EAFE Small-Cap ETF,2015-06-29,46435G839,US46435G8399,...,Small Cap,Broad,Global,https://www.ishares.com/us/products/273771/ish...,273771,8.564608e+07,ETF,IShares,NaN,NaN
209,HEZU,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI Eurozone ETF,2014-07-09,46434V639,US46434V6395,...,Large/Mid Cap,Broad,Europe,https://www.ishares.com/us/products/268708/ish...,268708,2.802581e+08,ETF,IShares,NaN,NaN
212,HEEM,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI Emerging Markets ETF,2014-09-23,46434G509,US46434G5099,...,Large/Mid Cap,Broad,Global,https://www.ishares.com/us/products/268704/ish...,268704,1.374384e+08,ETF,IShares,NaN,NaN
266,HEWG,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI Germany ETF,2014-01-31,46434V704,US46434V7047,...,Large/Mid Cap,Germany,Europe,https://www.ishares.com/us/products/259623/ish...,259623,3.606253e+07,ETF,IShares,NaN,NaN
267,HEFA,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI EAFE ETF,2014-01-31,46434V803,US46434V8037,...,Large/Mid Cap,Broad,Global,https://www.ishares.com/us/products/259622/ish...,259622,3.031149e+09,ETF,IShares,NaN,NaN
269,HEWJ,NaN,NaN,NaN,"Traceback (most recent call last):\n File ""pa...",ValueError,iShares Currency Hedged MSCI Japan ETF,2014-01-31,46434V886,US46434V8862,...,Large/Mid Cap,Japan,Asia Pacific,https://www.ishares.com/us/products/259624/ish...,259624,1.412138e+08,ETF,IShares,NaN,NaN


In [57]:
a = ETFScraper().query_holdings("HEWU", None)
a

INFO:etf_scraper.api:Querying for HEWU holdings as of latest from IShares
INFO:etf_scraper.scrapers:Querying IShares for HEWU holdings as of None
INFO:etf_scraper.scrapers:Found reported holdings date string Jan 20, 2023
INFO:etf_scraper.scrapers:Attempting to parse holdings data


,ticker,name,sector,asset_class,market_value,weight,notional_value,amount,price,location,exchange,currency,fx_rate,market_currency,Accrual Date,fund_ticker,as_of_date
0,EWU,ISHARES MSCI UNITED KINGDOM ETF,Financials,Equity,14590812.60,102.60,14590812.60,445385.0,32.76,United States,NYSE Arca,USD,1.00,USD,NaN,HEWU,2023-01-20
1,GBP,GBP/USD,Cash and/or Derivatives,Forwards,8548.20,0.06,8548.20,257000.0,1.00,United States,NaN,USD,0.81,USD,NaN,HEWU,2023-01-20
2,USD,USD CASH,Cash and/or Derivatives,Cash,8205.35,0.06,8205.35,8205.0,100.00,United States,NaN,USD,1.00,USD,NaN,HEWU,2023-01-20
3,GBP,GBP CASH,Cash and/or Derivatives,Cash,0.25,0.00,0.25,0.0,123.68,United Kingdom,NaN,USD,0.81,GBP,NaN,HEWU,2023-01-20
4,GBP,GBP/USD,Cash and/or Derivatives,Forwards,-2449.11,-0.02,-2449.11,-87000.0,1.00,United States,NaN,USD,0.81,USD,NaN,HEWU,2023-01-20
5,GBP,GBP/USD,Cash and/or Derivatives,Forwards,-384546.48,-2.70,-384546.48,-11545000.0,1.00,United States,NaN,USD,0.81,USD,NaN,HEWU,2023-01-20


In [17]:
s = a['ticker']=='DE'

In [20]:
import numpy as np
s.replace(False, np.nan).first_valid_index()

2

In [47]:
c

'iShares Currency Hedged MSCI United Kingdom ETF\nFund Holdings as of,"Jan 20, 2023"\nInception Date,"Jun 29, 2015"\nShares Outstanding,"550,000.00"\nStock,"-"\nBond,"-"\nCash,"-"\nOther,"-"\n\xa0\nTicker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date\n"EWU","ISHARES MSCI UNITED KINGDOM ETF","Financials","Equity","14,590,812.60","102.60","14,590,812.60","445,385.00","32.76","United States","NYSE Arca","USD","1.00","USD","-"\n"GBP","GBP/USD","Cash and/or Derivatives","Forwards","8,548.20","0.06","8,548.20","257,000.00","1.00","United States","-","USD","0.81","USD","-"\n"USD","USD CASH","Cash and/or Derivatives","Cash","8,205.35","0.06","8,205.35","8,205.00","100.00","United States","-","USD","1.00","USD","-"\n"GBP","GBP CASH","Cash and/or Derivatives","Cash","0.25","0.00","0.25","0.00","123.68","United Kingdom","-","USD","0.81","GBP","-"\n"GBP","GBP/USD","Cash and/or Derivatives","Forwards","-2,

In [44]:
c

'\ufeffiShares Currency Hedged MSCI United Kingdom ETF\nFund Holdings as of,"Jan 20, 2023"\nInception Date,"Jun 29, 2015"\nShares Outstanding,"550,000.00"\nStock,"-"\nBond,"-"\nCash,"-"\nOther,"-"\n\xa0\nTicker,Name,Sector,Asset Class,Market Value,Weight (%),Notional Value,Shares,Price,Location,Exchange,Currency,FX Rate,Market Currency,Accrual Date\n"EWU","ISHARES MSCI UNITED KINGDOM ETF","Financials","Equity","14,590,812.60","102.60","14,590,812.60","445,385.00","32.76","United States","NYSE Arca","USD","1.00","USD","-"\n"GBP","GBP/USD","Cash and/or Derivatives","Forwards","8,548.20","0.06","8,548.20","257,000.00","1.00","United States","-","USD","0.81","USD","-"\n"USD","USD CASH","Cash and/or Derivatives","Cash","8,205.35","0.06","8,205.35","8,205.00","100.00","United States","-","USD","1.00","USD","-"\n"GBP","GBP CASH","Cash and/or Derivatives","Cash","0.25","0.00","0.25","0.00","123.68","United Kingdom","-","USD","0.81","GBP","-"\n"GBP","GBP/USD","Cash and/or Derivatives","Forwards